In [ ]:
import os
import pandas as pd
import re
import numpy as np
from tqdm import tqdm

# Groq

In [ ]:
!pip install groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 2.3 MB/s eta 0:00:00


In [ ]:
from groq import Groq

client = Groq(
    api_key= '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',
    )

## Making a subset from the excel file from TextSegmentation notebook

In [ ]:
meds = pd.read_excel('Medicos_segmentados.xlsx')

In [ ]:
meds_1 = meds[0:3448]

In [ ]:
## Generating random numbers in order to make a question about the text
n = np.random.randint(low = 0,high=56,size=len(meds_1)).tolist()
numeros = [1 if i == 12 else 0 for i in n]

## Question Generation

In [ ]:
preguntas = []
for i in range(len(meds_1)):
    if numeros[i] == 1: ## Checks if the radom number is 1 to generate a question or not
        context = meds_1.texto[i]   ## Retrieve the context and generate a question about this text
        prompt =  f"""Poser une question sur ce texte donné {{context:}}. \n\n
                      Context: {context}?\n
                      Answer:
                      """
        respuesta = client.chat.completions.create(messages = [
                                                                {'role': 'system', 'content': 'pose une question dans le texte donné'},
                                                                {'role': 'user', 'content': prompt},
                                                                ],
                                                   model="llama3-8b-8192",
                                                   temperature=0.5,
                                                   max_tokens=1024,
                                                   top_p=1,
                                                   stop=None,
                                                   )
        salto = respuesta.choices[0].message.content.find('\n') ## To extract only the question and not the entire answer by the LLM
        preguntas.append(respuesta.choices[0].message.content[salto:].strip()) ## Appending the question to a list
    else:
        preguntas.append(0)

In [ ]:
## Appending the question as a column from the previous subset
meds_1['Pregunta'] = preguntas

In [ ]:
## Saving as a pickle to mantain all the data
meds_1.to_pickle("./meds_3448.pkl")